In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from dotenv import load_dotenv
load_dotenv()
# 설정
API_KEY          = os.getenv("GEMINI_API_KEY")
MODEL_ID         = "gemini-1.5-flash"
CSV_PATH         = "policy_actions.csv"
CONSUMPTION_PATH = "gemini_customer_response.txt"
TOP_K            = 3
EMBED_MODEL      = "all-MiniLM-L6-v2"


print("정책-시나리오 CSV 로드 중")
df = pd.read_csv(CSV_PATH)
policies = df["정책명"].unique()
policy_texts = {
    p: " ".join(df[df["정책명"] == p]["내용"].tolist())
    for p in policies
}
print(f"   정책 수: {len(policies)}")

print("소비 시나리오 로드 중")
with open(CONSUMPTION_PATH, "r", encoding="utf-8") as f:
    consumption_text = f.read().strip()
print(f"   소비 시나리오 길이: {len(consumption_text)} 문자")

print("문장 임베딩 생성 중")
embedder = SentenceTransformer(EMBED_MODEL)
policy_embeds     = embedder.encode(list(policy_texts.values()), convert_to_tensor=False)
consumption_embed = embedder.encode(consumption_text,           convert_to_tensor=False)

print("유사도 계산 및 상위 TOP_K 추출 중")
sims       = cosine_similarity([consumption_embed], policy_embeds)[0]
top_indices = sims.argsort()[::-1][:TOP_K]

print(f"상위 {TOP_K}개 정책")
for i, idx in enumerate(top_indices):
    print(f"   {i+1}. {policies[idx]}: {sims[idx]:.4f}")


# 가장 높은 유사도의 정책만 선택
best_idx = top_indices[0]
best_policy = policies[best_idx]
best_scen_text = policy_texts[best_policy]
best_score = float(sims[best_idx])

prompt = f"""소비 시나리오:
{consumption_text}

정책 시나리오:
{best_scen_text}

가장 유사도가 높게 나온 이유를 간결히 설명하고, 소비자에게 추천하세요"""


print(f"   모델: {MODEL_ID}")
print(f"   프롬프트 길이: {len(prompt)}자")

# Gemini API 호출
def call_gemini_api(prompt_text):


    # URL과 헤더 설정
    url = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_ID}:generateContent?key={API_KEY}"

    headers = {
        'Content-Type': 'application/json'
    }

    payload = {
        "contents": [
            {
                "parts": [
                    {"text": prompt_text}
                ]
            }
        ],
        "generationConfig": {
            "temperature": 0.1,
            "maxOutputTokens": 1024
        }
    }

    try:

        response = requests.post(url, headers=headers, json=payload, timeout=30)

        print(f" {response.status_code}")

        if response.status_code == 200:
            data = response.json()

            # 응답 구조 확인
            if 'candidates' in data and len(data['candidates']) > 0:
                candidate = data['candidates'][0]
                if 'content' in candidate and 'parts' in candidate['content']:
                    if len(candidate['content']['parts']) > 0:
                        result_text = candidate['content']['parts'][0].get('text', '').strip()
                        if result_text:
                            print(f"API 성공!")
                            print(f"응답: {result_text[:100]}...")
                            return result_text
                        else:
                            print("텍스트가 비어있음")
                            return "텍스트가 비어있음"
                    else:
                        print("parts가 비어있음")
                        return "parts가 비어있음"
                else:
                    print(" content 또는 parts 키가 없음")
                    return "content/parts 키 없음"
            else:
                print("candidates가 없거나 비어있음")
                print(f"실제 응답: {data}")
                return "candidates 없음"
        else:
            print(f"HTTP 오류: {response.status_code}")
            print(f"오류 내용: {response.text}")
            return f"HTTP {response.status_code} 오류"

    except requests.exceptions.Timeout:
        print(" 요청 타임아웃")
        return "타임아웃"
    except requests.exceptions.RequestException as e:
        print(f"요청 오류: {e}")
        return f"요청 오류: {e}"
    except Exception as e:
        print(f" 기타 오류: {e}")
        return f"기타 오류: {e}"

# API 호출 실행
evaluation_result = call_gemini_api(prompt)

# 모든 상위 정책을 결과에 추가
results = []
for i, idx in enumerate(top_indices):
    policy = policies[idx]
    score = float(sims[idx])

    if i == 0:  # 가장 높은 유사도
        cand = evaluation_result
    else:  # 나머지는 유사도만
        cand = "유사도만 계산됨"

    results.append({
        "정책명": policy,
        "유사도": score,
        "정합성 평가 및 이유": cand
    })

res_df = pd.DataFrame(results)
res_df.to_csv("alignment_results.csv", index=False, encoding="utf-8")
print("alignment_results.csv에 저장 완료")

print("\n결과 미리보기:")
print(res_df)

정책-시나리오 CSV 로드 중
   정책 수: 4
소비 시나리오 로드 중
   소비 시나리오 길이: 625 문자
문장 임베딩 생성 중


C:\Users\youngin\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


유사도 계산 및 상위 TOP_K 추출 중
상위 3개 정책
   1. ** KB맑은바다적금: 0.8333
   2. ** KB Green Wave 1.5 ℃ 정기예금: 0.8138
   3. KB Green Wave 1.5 ℃ 정기예금: 0.7986
   모델: gemini-1.5-flash
   프롬프트 길이: 1181자
 200
API 성공!
응답: 소비 시나리오와 정책 시나리오 모두 **'지속 가능한 소비'와 '환경 보호'를 중심으로 한 탄소 감축 활동**을 강조하고 있기 때문에 유사도가 높게 나타납니다.  정책 시나리오는 ...
alignment_results.csv에 저장 완료

결과 미리보기:
                           정책명       유사도  \
0                  ** KB맑은바다적금  0.833312   
1  ** KB Green Wave 1.5 ℃ 정기예금  0.813805   
2     KB Green Wave 1.5 ℃ 정기예금  0.798611   

                                         정합성 평가 및 이유  
0  소비 시나리오와 정책 시나리오 모두 **'지속 가능한 소비'와 '환경 보호'를 중심...  
1                                           유사도만 계산됨  
2                                           유사도만 계산됨  
